In [108]:
# sotmax Layerを用いたNNモデル
# 多層レイヤー（bitデータ(1を与えるとy20は1を返し、y21は反転を返す）
import numpy as np
class Layer:
    
    def __init__(self, m, n): 
        self._rate = 0.1
        self._weight = np.random.rand(m, n) - 0.5  # m行n列を設定(-0.5をすることで-0.5 ~ 0.5の範囲にする)
        self._bias = np.zeros((m, 1))  # m行1列の　0ベクトルを作成してくれている  # バイアスは出力の行
        self._y = None
        self._delta = None
        self._identity = np.identity(m)
        
        
    def set_state(self, x):
        """入力値を内積"""
        s = (self._weight.dot(x))+1*self._bias
        return s

    def activate(self, s):
        "今回はシグモイド関数"
        self._y = 1 / (1 + np.exp(-s))
        return self._y

    def softmax(self, s):
        self._y =  s * (self._identity.T - s)
        return self._y
                        
    
    def foward(self, x):
        """順伝播"""
        return self.activate(self.set_state(x))
    
    def back_propagation(self, prev_W, prev_delta, t):
        """勾配計算(誤差逆伝播)"""
        if prev_W is not None:
            # deltaを持って入ればパラメータの伝播( ∂E/∂W , ∂E/∂b)
            self._delta = prev_W.T.dot(prev_delta) * self._y  # (V00,01,10,11前回パラメータ) * (∂20,21前回delta) * (y10,y11今回出力)
        else:
            de = self._y - t
            dy = self._y * (np.identity(len(self._y)) - self._y).T
#             dy = self._y * (1 - self._y)  # シグモイドの偏微分
            self._delta = de * dy  # 初回誤差(delta) 
        return self._delta
    
    
    def update(self, input_x):
        """パラメーター更新"""
        self._weight -= self._rate * self._delta * input_x.T  # 入力値(y10を与えている) # self._delta * input_x.T(W00,01,10,11の∂)
        self._bias -= self._delta * self._bias
        return self._y
    
    def error_function(self, y, o):
        return (1/2 * np.power(y[0] - o[0],2)) + (1/2 * np.power(y[1] - o[1],2))

if __name__ == '__main__':
    bit_input = np.array([[[1.]],
                          [[0.]],
                          [[1.]],
                          [[1.]],
                          [[0.]]])
    bit_output = np.array([[[1.],
                            [0.]],
                           [[0.],
                            [1.]],
                           [[1.],
                            [0.]],
                           [[1.],
                            [0.]],
                           [[0.],
                            [1.]]])
    
    n1 = Layer(2,1) # １層目 ：(1入力)入力値が列、(2出力)出力値が行
    n2 = Layer(2,2) # ２層目 ：(2入力)入力値が列、(2出力)出力値が行
    prev_W = None
    prev_delta = None
    for _ in range(100):
        for input_x, anser in zip(bit_input,bit_output):
            n1_myself_y = n1.foward(input_x)  # 1層目に値実行
            n2.foward(n1_myself_y)  # 2層目に値実行
            n2.back_propagation(prev_W, prev_delta, anser)  # 2層目に前の重みV と 前のdeltaを付けて実行
            prev_W, prev_delta = n2._weight, n2._delta  # 2層目の重みV と deltaを抜き出している(後に利用するため)
            n1_delta = n1.back_propagation(prev_W, prev_delta, anser)  # 1層目に2層目の重みV とdeltaを付けて実行

            n2.update(n1_myself_y) # 2層目のパラメータの更新
            n1.update(input_x) # 1層目のパラメータの更新
            
            n1_myself_y = n1.foward(input_x) # 最終的な誤差を計算するために一度foward n1から
            n2_myself_y = n2.softmax(n1_myself_y)  # n2は最後なのでsoftmaxへ
            
            print("誤差：", n1.error_function(n2_myself_y, anser)) # n1.fowardから求められた予測値

ValueError: non-broadcastable output operand with shape (2,1) doesn't match the broadcast shape (2,2)